In [1]:
from langchain_community.document_loaders import TextLoader,WebBaseLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
import bs4

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:

# Load text from a Wiki pedia
web = WebBaseLoader(web_paths=("https://en.wikipedia.org/wiki/Car",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer("p")))
doc_web = web.load()

In [3]:
# Character text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs_chunk = text_splitter.split_documents(doc_web)   

Created a chunk of size 34493, which is longer than the specified 1000


In [4]:
#Embedding Model
embeddings = OllamaEmbeddings(model="llama3.2")
db = FAISS.from_documents(docs_chunk, embeddings)

C:\Users\Kavin\AppData\Local\Temp\ipykernel_25388\2992112148.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="llama3.2")


In [5]:
db

In [6]:
query = "petrol car "
docs = db.similarity_search(query)
docs[0]

Document(id='f5a6e3f6-8f0c-47df-8808-eaa71235cb02', metadata={'source': 'https://en.wikipedia.org/wiki/Car'}, page_content='Effects:\n\nMitigation:')

As a Retreiver

We can also convert the vectorstore into a retriver class. this allows us to easily usee it in other Lang Chain methods, Which largely work with retreivers

In [7]:
retreiver = db.as_retriever()
docs = retreiver.invoke(query)
docs[0].page_content

'Effects:\n\nMitigation:'

In [8]:
docs_score = db.similarity_search_with_score(query)
docs_score

[(Document(id='f5a6e3f6-8f0c-47df-8808-eaa71235cb02', metadata={'source': 'https://en.wikipedia.org/wiki/Car'}, page_content='Effects:\n\nMitigation:'),
  np.float32(7743.2715)),
 (Document(id='08517335-b2e1-4db5-9040-a33f0b684cf1', metadata={'source': 'https://en.wikipedia.org/wiki/Car'}, page_content='A car, or an automobile, is a motor vehicle with wheels. Most definitions of cars state that they run primarily on roads, seat one to eight people, have four wheels, and mainly transport people rather than cargo.[1][2] There are around one billion cars in use worldwide.[citation needed]\nThe French inventor Nicolas-Joseph Cugnot built the first steam-powered road vehicle in 1769, while the Swiss inventor François Isaac de Rivaz designed and constructed the first internal combustion-powered automobile in 1808. The modern car—a practical, marketable automobile for everyday use—was invented in 1886, when the German inventor Carl Benz patented his Benz Patent-Motorwagen. Commercial cars bec

In [9]:
#Save the vector store to local disk

db.save_local("faiss_car")    

In [10]:
#Load the vector store from local disk
new_db = FAISS.load_local("faiss_car", embeddings,allow_dangerous_deserialization=True)